# Tarea
## Modelación Supervisada
Esta tarea consiste de dos partes. La primera consiste en la primera mitad del curso, y la segunda en la otra.
Esta tarea vale 100 puntos y contiene varias secciones donde podrás ganar puntos extras. Justifica tus respuestas y respeta el deadline. Sigue las instrucciones en los comentarios como guía.

**Deadline: 20 marzo 23:59 PM**

# Parte 1 - Exploración de datos y tus primeros modelos supervisados. [30 puntos]
Esta parte de la tarea consiste en refinar tus skills de Data Exploration y Pre-processing. También podrás entrenar tus primeros modelos supervisados. 

In [1]:
## Importa librerías útiles
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import itertools
import random


# Sklearn
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, roc_curve, confusion_matrix, roc_auc_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import preprocessing

In [2]:
# Funciones útiles
def grafica_matriz_confusion(cm, classes=[0,1], normalize=False, title='Matriz de confusión\n', cmap=plt.cm.Reds):
    """ 
    Función para mostrar la matriz de confusión de un problema de clasificación binario. 
    El parámetro cm puede ser el objeto resultante de ejecutar la función confusion_matrix() de la librería sklearn.
    
    Args:
        cm         matriz de confusión de sklearn
        classes    categorías de la variable target
        normalize  normaliza asignando el parámetro True
        title      string para definir el título para la gráfica
        cmap       mapa de colores
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=0)
    plt.yticks(tick_marks, classes)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    thresh = cm.max()
    text = [["VN =", "FN ="], ["FP =", "VP ="]]
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, "     {}".format(round (cm[i, j],2)), horizontalalignment="left",
                 color="white" if cm[i, j] > thresh else "black")
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, text[j][i] , horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('Valor real')
    plt.xlabel('Predicción')
    
def grafica_curva_roc(fpr, tpr, title='Curva ROC', note=''):
    """
    Función para graficar la curva ROC
    Los parámetros fpr y tpr son el output de ejecutar la función roc_curve de sklearn
    
    Args:
        fpr        Tasa de falsos positivos 
        tpr        Tasa de verdaderos positivos
        title      sting para definir el título de la gráfica
        note       Nota para mostrar en la gráfica
    """
    plt.figure(1)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.plot(fpr, tpr)
    plt.xlabel('Tasa de falsos positivos ')
    plt.ylabel('Tasa de verdaderos positivos')
    plt.title(title)
    if note: plt.text(0.6, 0.2, note)
    plt.show()

## 1) Exploración y Limpieza de datos [10 puntos]
Tu tarea aquí es explorar los data sets **happiness2020.csv** y **countries_info.csv** y responder algunas preguntas.

El layout de **happiness2020.csv** es:

* country: Nombre del país
* happiness_score: Score de felicidad
* social_support: Ayuda social
* healthy_life_expectancy: Expectativa de vida
* freedom_of_choices: Libertar de tomar decisiones de vida
* generosity: Generosidad
* perception_of_corruption: Percepción de la corrupción
* world_region: Región en donde se encuentra el país en cuestión


El layout de **countries_info.csv**

* country_name: Nombre del país
* area: Área en metros cuadrados
* population: Número de personas
* literacy: Porcentaje de alfabetismo


In [3]:
# Carga el data set happiness2020.csv
happi = pd.read_csv('happiness2020.csv')
happi

,country,happiness_score,social_support,healthy_life_expectancy,freedom_of_choices,generosity,perception_of_corruption,world_region
0,Afghanistan,2.5669,0.470367,52.590000,0.396573,-0.096429,0.933687,South Asia
1,Albania,4.8827,0.671070,68.708138,0.781994,-0.042309,0.896304,Central and Eastern Europe
2,Algeria,5.0051,0.803385,65.905174,0.466611,-0.121105,0.735485,Middle East and North Africa
3,Argentina,5.9747,0.900568,68.803802,0.831132,-0.194914,0.842010,Latin America and Caribbean
4,Armenia,4.6768,0.757479,66.750656,0.712018,-0.138780,0.773545,Commonwealth of Independent States
...,...,...,...,...,...,...,...,...
130,Venezuela,5.0532,0.890408,66.505341,0.623278,-0.169091,0.837038,Latin America and Caribbean
131,Vietnam,5.3535,0.849987,67.952736,0.939593,-0.094533,0.796421,Southeast Asia
132,Yemen,3.5274,0.817981,56.727283,0.599920,-0.157735,0.800288,Middle East and North Africa
133,Zambia,3.7594,0.698824,55.299377,0.806500,0.078037,0.801290,Sub-Saharan Africa


In [4]:
# Explora el set de datos (número de variables, observaciones, checa 5 registros y más!)
# Número de variables, observaciones,
tamano = happi.shape
print('El df tiene {} filas y {} columnas'.format(tamano[0], tamano[1]))

El df tiene 135 filas y 8 columnas


In [5]:
# Tipos de datos por columnas
happi.dtypes

country                      object
happiness_score             float64
social_support              float64
healthy_life_expectancy     float64
freedom_of_choices          float64
generosity                  float64
perception_of_corruption    float64
world_region                 object
dtype: object

In [6]:
# Estadisticos descriptivos
happi.describe()

,happiness_score,social_support,healthy_life_expectancy,freedom_of_choices,generosity,perception_of_corruption
count,135.000000,135.000000,135.000000,135.000000,135.000000,135.000000
mean,5.525062,0.815165,64.762495,0.790657,-0.022749,0.727447
std,1.123414,0.116311,6.694776,0.117230,0.146165,0.180406
min,2.566900,0.468671,48.003624,0.396573,-0.300907,0.109784
25%,4.749000,0.740405,59.809444,0.720287,-0.130953,0.675699
50%,5.541500,0.836419,66.480164,0.811602,-0.042309,0.780623
75%,6.292700,0.910313,69.145870,0.886453,0.083279,0.848558
max,7.808700,0.974670,76.804581,0.974998,0.519587,0.935585


In [7]:
# 5 registros
happi.head(5)

,country,happiness_score,social_support,healthy_life_expectancy,freedom_of_choices,generosity,perception_of_corruption,world_region
0,Afghanistan,2.5669,0.470367,52.590000,0.396573,-0.096429,0.933687,South Asia
1,Albania,4.8827,0.671070,68.708138,0.781994,-0.042309,0.896304,Central and Eastern Europe
2,Algeria,5.0051,0.803385,65.905174,0.466611,-0.121105,0.735485,Middle East and North Africa
3,Argentina,5.9747,0.900568,68.803802,0.831132,-0.194914,0.842010,Latin America and Caribbean
4,Armenia,4.6768,0.757479,66.750656,0.712018,-0.138780,0.773545,Commonwealth of Independent States


In [9]:
# Contabilizar los duplicados
happi[happi.duplicated()].shape

(0, 8)

* No existen dupliucados

In [13]:
# Funcion para valores inexistentes

def valores_inexistentes(df):
    n_records = len(df)
    
    for columna in df:
        print('{} | {} | {}'.format(df[columna].name,
                                    len(df[df[columna].isnull()]) / (1.0*n_records),
                                    df[columna].dtype))

# Valores inexistentes    
valores_inexistentes(happi)   

country | 0.0 | object
happiness_score | 0.0 | float64
social_support | 0.0 | float64
healthy_life_expectancy | 0.0 | float64
freedom_of_choices | 0.0 | float64
generosity | 0.0 | float64
perception_of_corruption | 0.0 | float64
world_region | 0.0 | object


* No tenemos valores inexistentes.

In [35]:
# Carga el data set countries_info.csv
# La variable literacy contiene números con formato europeo. Hint: busca en la documentación de pandas como importar estos datos para ser interpretados como números
country = pd.read_csv('countries_info.csv', decimal=',')
country

,country_name,area,population,literacy
0,afghanistan,647500,31056997,36.0
1,albania,28748,3581655,86.5
2,algeria,2381740,32930091,70.0
3,argentina,2766890,39921833,97.1
4,armenia,29800,2976372,98.6
...,...,...,...,...
130,venezuela,912050,25730435,93.4
131,vietnam,329560,84402966,90.3
132,yemen,527970,21456188,50.2
133,zambia,752614,11502010,80.6


In [36]:
# Explora el set de datos (número de variables, observaciones, checa 5 registros y más!)
#Número de variables, observaciones,
tamano = country.shape
print('El df tiene {} filas y {} columnas'.format(tamano[0], tamano[1]))

El df tiene 135 filas y 4 columnas


In [37]:
#Tipos de datos por columnas
country.dtypes

country_name     object
area              int64
population        int64
literacy        float64
dtype: object

In [38]:
#Estadisticos descriptivos
country.describe()

,area,population,literacy
count,1.350000e+02,1.350000e+02,133.000000
mean,9.007829e+05,4.552204e+07,81.851128
std,2.244994e+06,1.505270e+08,20.514483
min,3.160000e+02,2.993880e+05,17.600000
25%,6.540500e+04,4.636146e+06,70.000000
50%,2.375000e+05,1.023546e+07,90.900000
75%,7.000570e+05,2.967980e+07,98.400000
max,1.707520e+07,1.313974e+09,100.000000


In [39]:
#5 registros
country.head(5)

,country_name,area,population,literacy
0,afghanistan,647500,31056997,36.0
1,albania,28748,3581655,86.5
2,algeria,2381740,32930091,70.0
3,argentina,2766890,39921833,97.1
4,armenia,29800,2976372,98.6


In [40]:
# Contabilizar los duplicados
country[country.duplicated()].shape


(0, 4)

* No existen dulicados

In [41]:
# Valores inexistentes
valores_inexistentes(country)

country_name | 0.0 | object
area | 0.0 | int64
population | 0.0 | int64
literacy | 0.014814814814814815 | float64


* Existen valores inexistentes en la columna 'literacy', estos son muy pocos.

In [43]:
# Une los datos utilizando la función merge de pandas
# Ojo: ¿Cuál es la llave de cruce en cada set de datos? ¿Ves algo que pueda afectar a la hora de cruzar los datos con esta llave? Explica por que y toma la decisión correcta para transformar una de las variables de cruce
country = country.rename(columns={'country_name': 'country'})
country['country'] = country['country'].str.capitalize()
country

,country,area,population,literacy
0,Afghanistan,647500,31056997,36.0
1,Albania,28748,3581655,86.5
2,Algeria,2381740,32930091,70.0
3,Argentina,2766890,39921833,97.1
4,Armenia,29800,2976372,98.6
...,...,...,...,...
130,Venezuela,912050,25730435,93.4
131,Vietnam,329560,84402966,90.3
132,Yemen,527970,21456188,50.2
133,Zambia,752614,11502010,80.6


* La llave de cruze es la columna country, el problema es que al no tener el mismo nombre y el mismo formatos no sera posible realizar el merge.
* Se renombro la columna 'country_name' a 'country' del dataset country, posteriormente se utiizo 'capitalize' para tener el mismo formato de la columna 'country del dataset happi

In [45]:
dftotal = happi.merge(country, how='inner', on='country')
dftotal

,country,happiness_score,social_support,healthy_life_expectancy,freedom_of_choices,generosity,perception_of_corruption,world_region,area,population,literacy
0,Afghanistan,2.5669,0.470367,52.590000,0.396573,-0.096429,0.933687,South Asia,647500,31056997,36.0
1,Albania,4.8827,0.671070,68.708138,0.781994,-0.042309,0.896304,Central and Eastern Europe,28748,3581655,86.5
2,Algeria,5.0051,0.803385,65.905174,0.466611,-0.121105,0.735485,Middle East and North Africa,2381740,32930091,70.0
3,Argentina,5.9747,0.900568,68.803802,0.831132,-0.194914,0.842010,Latin America and Caribbean,2766890,39921833,97.1
4,Armenia,4.6768,0.757479,66.750656,0.712018,-0.138780,0.773545,Commonwealth of Independent States,29800,2976372,98.6
...,...,...,...,...,...,...,...,...,...,...,...
117,Venezuela,5.0532,0.890408,66.505341,0.623278,-0.169091,0.837038,Latin America and Caribbean,912050,25730435,93.4
118,Vietnam,5.3535,0.849987,67.952736,0.939593,-0.094533,0.796421,Southeast Asia,329560,84402966,90.3
119,Yemen,3.5274,0.817981,56.727283,0.599920,-0.157735,0.800288,Middle East and North Africa,527970,21456188,50.2
120,Zambia,3.7594,0.698824,55.299377,0.806500,0.078037,0.801290,Sub-Saharan Africa,752614,11502010,80.6


In [ ]:
dftotal.dtypes

In [ ]:
# Explora el set de datos que contiene los dos sets anteriores (número de variables, observaciones, checa 5 registros y más!)


In [ ]:
# Determina el top 10 de países más felices


In [ ]:
# Determina el top 10 de regiones con países más felices (¿Cómo puedes medir la felicidad por región?)


In [ ]:
# ¿Cuáles son los países de la región con países más felices? Muestra el nombre y el score de felicidad


In [ ]:
# ¿Cuál es el top 10 de países con mayor porcentaje de alfabetismo?


In [ ]:
# ¿Cuáles son las regiones del mundo que tienen los países con mayor tasa de alfabetismo?


In [ ]:
# Muestra los países de la región con mayor porcentaje de alfabetismo


In [ ]:
# ¿Cuál es el top 10 de países de países con menor porcentaje de alfabetismo?


In [ ]:
# ¿Cuáles son las regiones del mundo que tienen los países con menor tasa de alfabetismo?


In [ ]:
# Muestra los países de la región con menor porcentaje de alfabetismo


In [ ]:
# Determina el top 10 de países con mayor cantidad de personas


In [ ]:
# ¿Cuáles son las regiones más pobladas del mundo?


In [ ]:
# ¿Cuáles son los países en las regiones menos pobladas del mundo?


In [ ]:
# ¿Cuál es el score de felicidad de los tres países más poblados del mundo?


In [ ]:
# ¿Cuál es el score de felicidad de los tres países menos poblados del mundo?


In [ ]:
# Gráfica y explora las variables happiness_score, literacy y population
# Hint: Utiliza los métodos vistos en clase para variables númericas

In [ ]:
# ¿Existe una relación (¿de qué tipo?) entre la variable happiness_score y healthy_life_expectancy?
# Hint: Gráfica la relación que existe entre las variables


¿Podrías entrenar un modelo de regresión lineal para predecir la variable healthy_life_expectancy con base en la variable independiente happiness_score? ¿Sería un buen modelo para predecir la expectativa de vida? Elabora tu respuesta


In [ ]:
# Define la variable target (healthy_life_expectancy)


# Define el conjunto de variables independientes


# Utiliza la función de sklearn para partir los datos en set de entrenamiento y validación
# 70% train, 30% test, utiliza una semilla para poder replicar resultados (busca en la documentación de sklearn como hacer esto)


In [ ]:
# Entrena un modelo de regresión lineal y determina el score R² y el score de error (MSE)


## 2) Regresión logísitica utilizando el método de gradiente descendente aplicado a una función de costo de error cuadrático medio [20 puntos]
En esta parte se requiere llevar a cabo una tarea de clasificación. Para ello vamos a utilizar una regresión logística. Primero que nada expliquemos el modelo anterior.

**Regresión logística**
Es un modelo que sirve para clasificar observaciones. En pocas palabras (y para esta tarea) la regresión logística es una transformación del modelo de regresión lineal sin término regularizador. Es decir, una regresión logística transforma las predicciones de una regresión lineal a un valor de probabilidad. 

$y = \textbf{X}^T\textbf{W}$

$y\_pred = \sigma(y)$

En esta parte de la tarea deberás utilizar el método de gradiente descendente para estimar los parámetros de una regresión lineal y después transformar las predicciones en probabilidades utilizando la función sigmoide. 

Vamos a utilizar un data set con información de los pasajeros del barco Titanic (**titanic.xls**). A continuación puedes ver el layout:

* name: Nombre del pasajero
* sex: Sexo del pasajero
* age: edad en años
* sibsp: # hijos / esposos abordo del Titanic
* parch: # padres / niños abordo del Titanic
* ticket: número de ticket de pasajero
* fare: precio del ticket
* cabin: Número de cabina en el barco
* embarked: Puerto de embarcación


In [ ]:
# Carga los datos


In [ ]:
# Explora el data set
# Aquí ocupa todo lo que ya sabes para analizar un set de datos. Si el análisis que vas a realizar es muy bueno, podríamos considerar tener puntos extras


In [ ]:
# ¿Hace sentido utilizar todas las columnas para entrenar un modelo de clasificación? Elimina variables que podrían meter ruido y quedate solo con las que consideres cruciales


In [ ]:
# Preprocesa los datos
# Hint: crea variables dummies para la variable "sex"


In [ ]:
# Mostrar el nombre de las variables dummy creadas


In [ ]:
# Define la variable target (survived)


# Define el conjunto de variables independientes


In [ ]:
# Determina cuántas observaciones tienen un valor missing


In [ ]:
# Cuál o cuáles variables tienen valores missing


In [ ]:
# Describe las variables para ver qué método podría servirnos para imputar valores nulos


In [ ]:
# Imputa los valores missing de las variables con valores missing


# Comprueba que ya no existan registros con valores nulos


In [ ]:
# Revisa los rangos de las distintas variables independientes


In [ ]:
# Normaliza los valores de las variables independientes


In [ ]:
# Revisa los rangos de las distintas variables independientes


In [ ]:
# Utiliza la función de sklearn para partir los datos en set de entrenamiento y validación
# 70% train, 30% test, utiliza una semilla para poder replicar resultados (busca en la documentación de sklearn como hacer esto)


# Define las Matrices de variables independientes con intercepto (agregar una columna con puros 1s)


In [ ]:
# Define todas las funciones que necesitas para llevar a cabo el proceso de gradiente descendente con una función de costo de error cuadrático medio
# Determina los parámetros W tal óptimos. Para ello encuentra los hyper parámetros óptimos para la tasa de aprendizaje gamma y el número máximo de iteraciones
# Recuerda agregar una columna de 1s a la matrix X, con ello estarás optimizando todos los parámetros, incluido el intercepto o w0


Ahora vamos a transformar los resultados del proceso anterior en probabilidades para el modelo de clasificación. Para ello debes expresar la variable target como una combinación lineal de los parámetros óptimos obtenidos con el proceso anterior y transformarlos utilizando la función sigmoide

$\sigma(t) = \displaystyle \frac{1}{1+exp(-t)}$

In [ ]:
# Define la función sigmoide


In [ ]:
# Determina y_pred
# Hint: Recuerda que en una regresión lineal, la variable target se puede escribir como combinación lineal de los parámetros óptimos y la matrix X (y = XW^T)
# Utiliza X_b (X con una fila de 1s para representar el intercepto)


# Transforma el resultado anterior utilizando la función sigmoide definida anteriomente


In [ ]:
# Asignar una etiqueta positiva o negativa si la probabilidad sobrepasa un threshold de probabilidad determinado
# Prueba con threshold de probabilidad = 60%


**[Stretch: Por 5 puntos extras]** Determinar cuál es el valor óptimo para el threshold de probabilidad

In [ ]:
# Ya sea con 60% como threshold de probabilidad u otro valor optimizado, evalúa el modelo creado
# Distribución de predicciones


In [ ]:
# Muestra la matriz de confusión


In [ ]:
# Cuántos errores cometimos?


In [ ]:
# Calcula la efectividad


In [ ]:
# Calcula el recall y la precisión de la predicción


In [ ]:
# Utiliza la función de sklearn llamada roc_curve para determinar la lista de tasa de falsos y verdaderos positivos


# Parte 2 - Entrena un modelo de clasificación de sentimientos. [70 puntos]
Esta parte de la tarea consiste en entrenar un modelo de clasificación de sentimientos. Lee las instrucciones con cuidado y justifica tus resultados.

## Descripción
En esta tarea vamos a ayudar a un científic@ de datos a predecir el sentimiento de reseñas que los usuarios de la aplicación móvil de la Institución Financiera para la que trabaja dan acerca de ella. Para ello, tiene un set de datos (**data_app_movil.csv**) con el siguiente layout:

**Layout Tabla de datos*:**
* at: Fecha en la que la reseña fue hecha
* content: Texto de la reseña
* repliedAt: Fecha en la que se respondió a la reseña hecha por el usuario
* replyContent: Texto de la respuesta a la reseña hecha por el usuario
* reviewCreatedVersion: Versión de la aplicación móvil que el usuario tenía al dar la reseña de la misma
* reviewId: ID del usuario que escribió la reseña de la aplicación móvil
* thumbsUpCount: Número de pulgares hacía arriba (votos de otros usuarios que apoyan una reseña)
* sentiment: Sentimiento de la reseña escrita por un usuario

**Objetivo:**
En esta tarea deberás ayudar al científic@ de datos a entrenar un modelo supervisado para clasificar los sentimientos de las reseñas de los usuarios de la aplicación móvil.

*Los datos con los que fue hecho este set son públicos

## Importa librerías importantes

In [ ]:
# Import useful libraries
import matplotlib.pyplot as plt
import seaborn as sns
import statistics as sts
from scipy import stats
import re
import pandas as pd
import numpy as np
from numpy import zeros
from numpy import asarray
from numpy import array
import itertools
import collections
import csv

# Tunning plots
import matplotlib.lines as mlines
from matplotlib.ticker import MaxNLocator
from wordcloud import WordCloud, ImageColorGenerator
%matplotlib inline
pd.set_option('display.max_columns', 100)
SEED = 15432

# scikit-learn
from sklearn.linear_model            import LinearRegression, LogisticRegression, Ridge, RidgeCV
from sklearn.svm                     import SVC
from sklearn.ensemble                import RandomForestClassifier
from sklearn.ensemble                import GradientBoostingClassifier
from sklearn.preprocessing           import LabelEncoder,OneHotEncoder
from sklearn.model_selection         import StratifiedKFold
from sklearn.model_selection         import validation_curve
from sklearn.model_selection         import train_test_split
from sklearn.metrics                 import confusion_matrix
from sklearn.metrics                 import accuracy_score
from sklearn.model_selection         import train_test_split
from sklearn.model_selection         import cross_val_predict
from sklearn.model_selection         import cross_val_score
from sklearn                         import metrics
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition           import TruncatedSVD
from sklearn.naive_bayes             import GaussianNB
from sklearn.metrics         import roc_curve, roc_auc_score

# Ignore some warnings 
import warnings
warnings.filterwarnings('ignore')

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils  import simple_preprocess
from gensim.models import CoherenceModel

# NLTK
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import WordPunctTokenizer
from nltk.stem     import WordNetLemmatizer 
from nltk.corpus   import stopwords

# Read/Write xlsx files
from openpyxl import workbook 
from openpyxl import load_workbook
import os

## Importa los datos
**Ojo:** importa solo las variables que necesites para la tarea (layout). Intenta nombrar al dataframe **data** 

## Exploración Básica de Datos [10 puntos]
En esta sección tienes que explorar los datos, haz un análisis exhaustivo para responder las siguientes preguntas:

* ¿Cuántos registros y columnas tiene el set de datos?
* ¿Qué porcentaje de valores nulos presenta cada variable? 
* ¿Hay reseñas duplicadas? ¿Cuáles son?
* ¿Cuál es la variable target?
* ¿Cuál es la distribución de la variable target?
* ¿Cuál es la reseña con más apoyo por parte de otros usuarios (thumbsUpCount)?
* ¿Qué porcentaje de reseñas son respondidas por la Institución financiera?
* ¿Cuál es el rango de fechas en las que fueron escritas las reseñas de este set de datos?
* ¿Qué otras cosas básicas crees que vale la pena analizar de este set de datos?

## Procesamiento de Texto [5 puntos]
Para que un modelo de predicción pueda recibir como input un corpus de texto, primero se tiene que limpiar, y luego convertr a una matriz numérica tal que conserve la semántica del texto. 
En esta sección haremos la limpieza de la variable de texto llamada 'content'. Existen varias formas de hacerlo, enlistamos las técnicas que usaremos:
* Eliminar caracteres especiales, signos de puntiación, números
* Eliminar acentos
* Convertir a minúsculas todas las palabras en el texto
* Eliminar stop words ¿Qué son las stop words? Investiga en internet y escribe un parráfo de ello
* Eliminar palabras que tengan menos de 3 caracteres

In [ ]:
# Define funciones para limpiar texto
# Importa algunas librerías útiles para las tareas de limpieza de texto
import nltk
import spacy
from spacy.lang.es import Spanish
import re
import string
from unidecode import unidecode
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import itertools
import collections
import csv

# Si no has descargado el archivo de stop words en español, al ejecutar este pesado de código te saldrá un error. 
# Investiga en internet cómo descargar la lista de stopwords en español
nlp = Spanish()

# Stop words en español
stop_words = list(set(stopwords.words('spanish')))

# ¿Con el conocimiento que tienes de stop words, cuáles crees que nos falten de agregar? Yo te agrego unas, pero piensa en otras
stop_words.append('aplicacion')
stop_words.append('banco')

# Imprime la longitud de la lista de stopwords en español
print("The stop words list has a length of {} words".format(len(stop_words)))

In [ ]:
# Define funciones para limpiar texto
def remove_accents(a):
    """ Función para eliminar acentos
      
        Args:  
        a pandas series string
    """
    return unidecode(a)

def clean_re(txt):
    """ Función para eliminar puntuación, convertir a minusculas, entre otras cosas
      
        Args:  
        txt pandas series string
    """
    # Convierte a minusculas
    txt = pd.Series(txt).str.lower()
    
    # Elimina caracteres especiales de tipo \t\n\r\f\v
    txt = pd.Series(txt).apply(lambda x: re.sub(r'[^\w\s]',' ',x))
    
    # Elimina palabras con menos de tres letras
    txt = txt.apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
    
    # Elimina números
    txt = pd.Series(txt).apply(lambda x: re.sub(r'\b\d+(?:\.\d+)?\s+', ' ', x))
    
    return txt

def remove_stopwords(txt):
    """ Función para eliminar stop words
      
        Args:  
        txt pandas series string
    """
    txt = pd.Series(txt).apply(lambda x: ' '.join([item for item in x.split() if item not in stop_words]))
    return txt

In [ ]:
# Aplica las funciones a la variable 'content'
# Esta parte del código tarda un poco
data['content_clean'] = data['content'].apply(clean_re)
data['content_clean'] = data['content_clean'].apply(remove_accents)
data['content_clean'] = data['content_clean'].apply(remove_stopwords)

**Revisa** la variable 'content_clean'
* ¿Cómo se diferencia de la variable 'content'?
* ¿De qué otras maneras podrías limpiar la variable 'content' para que el texto aporte el más valor posible al modelo de predicción?

In [ ]:
# Muestra los primeros 5 registros del data set con las variables 'content' y 'content_clean'


In [ ]:
# Investiga en internet qué es tokenizar una oración y entiende que hace la siguiente línea de código
data['list_of_words'] = list(data['content_clean'].apply(lambda x : word_tokenize(x))) 

In [ ]:
# Muestra los primeros 5 registros del data set con las variables 'content', 'content_clean', y 'list_of_words'


## Exploración avanzada [10 puntos]
En esta parte analizamos de manera más avanzada la variable de texto. Investiga como dar respuesta  a las siguientes preguntas:
* Cuál es la distribución de la longitud de reseñas. Hint: Calcula la longitud de cada reseña en el set de datos. Saca estadísticos principales (media, mediana, desviación estándar, moda). Gráfica la distribución 
* ¿Cuál es el top 10 de palabras más usadas para las reseñas no negativas?
* ¿Cuál es el top 10 de palabras más usadas para las reseñas negativas?
* Investiga como hacer una nube de palabras y crea una para las palabras más usadas en todo el set de datos 

## Procesamiento de datos [5 puntos]
En esta parte tendrás que preparar los datos para poder entrenar modelos de aprendizaje de máquina. Para ello, tendrás que hacer lo siguiente:
* Separa el set de datos en set de variables independientes (X) y en variable target (y)
* Divide el set de datos en train (70%) y test (30%). Utiliza la semilla SEED=12345 para poder reproducir tus resultados

## Word Embeddings [5 puntos]
Convierte los datos de texto en una matriz numérica que los modelos de aprendizaje de máquina puedan utilizar. Para ello, te apoyaré con unas líneas de código pero te corresponde a ti lo siguiente:
* Investigar qué es un word embedding y por qué son importantes para entrenar un modelo de clasificación cuando se utilizan datos de texto
* Investigar acerca de los word embeddings TF-IDF y Bag of Words (BoW)

Yo te voy a enseñar cómo convertir los datos, pero te corresponde entender cómo funcionan y cómo manipularlos. 

In [ ]:
# Define función útil
def text_transformer(X_train, X_test, vectorizer):
    """ Función para crear word embeddings (TF-IDF y BoW)
      
        Args:  
        X_train pandas dataframe
        X_test  pandas dataframe
        varToVector string que contiene nombre de variable de texto
        vectorizer objeto para crear un word embedding de tipo TF-IDF y BoW
    """
    # Entrena método para convertir de texto a matrix numérica
    vectorizer_ = vectorizer
    vectorizer_.fit(X_train)

    X_train = vectorizer_.transform(X_train)
    X_test = vectorizer_.transform(X_test)
    return X_train, X_test

**TF-IDF**

Investiga qué parámetros puede tomar e intenta optimizarlos. X_train_TFID y X_test_TFID serán los sets de datos que tendrás que utilizar para entrenar modelos en la sección siguiente

In [ ]:
# Crea un word embedding de tipo TF-IDF con el objeto de Sklearn llamado TfidfVectorizer
# TfidfVectorizer(Por optimizar)
X_train_TFID, X_test_TFID = text_transformer(X_train, X_test, TfidfVectorizer(#Optimiza algunos parámetros))

# Convierte el resultado anterior en una matriz poco densa que pueda ser usada para entrenar un modelo
X_train_TFID = pd.DataFrame(X_train_TFID.toarray())
X_test_TFID = pd.DataFrame(X_test_TFID.toarray())

**Bag of Words (BoW)**

Investiga qué parámetros puede tomar e intenta optimizarlos. X_train_BoW y X_test_BoW serán los sets de datos que tendrás que utilizar para entrenar modelos en la sección siguiente

**[Stretch: 5 puntos extras] Word embeddings más complejos**

Investiga técnicas más complejas para crear word embeddings y utilizalas en la sección de entrenamiento de modelos. ¿Crees que ayuden a mejorar el performance de los mismos?

## Entrena modelos de aprendizaje de máquina [25 puntos]
Entrena al menos tres tipos de modelos de clasificación. Todo modelo extra podrá ser considerado para puntos extras. Tienes la libertad de entrenar los modelos de la manera que desees, entre mejor sea la metodología de entrenamiento y evaluación de resultados, más puntos obtendrás. 

**[Stretch: 10 puntos extras]** Para todos los modelos que entrenes, determinar cuál es el valor óptimo para el threshold de probabilidad

## Selecciona el mejor modelo [10 puntos]
En esta parte debes seleccionar el mejor modelo, ya sea usando bootstrap o de la manera que creas adeacuada. Justifica bien tus decisiones